## 펑션 콜링(function calling) 작동 방식 이해하기 

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool
import numpy as np

llm=OpenAI(model="gpt-4o")

In [4]:
def add(a,b):
    """ 주어진 두 숫자를 더하고 결과를 출력합니다 """
    return a+b


def mul(a,b):
    """ 주어진 두 숫자를 곱하고 결과를 출력합니다 """
    return a*b

def div(a,b):
    """ 주어진 두 숫자 중 첫번 째 숫자를 두번 째 숫자로 나누고 결과를 출력합니다 """
    return a/b


In [5]:
at=FunctionTool.from_defaults(fn=add)
mt=FunctionTool.from_defaults(fn=mul)
dt=FunctionTool.from_defaults(fn=div)

In [6]:
agent_worker=FunctionCallingAgent.from_tools([at,mt,dt], 
                                            llm=llm, 
                                            verbose=True,
                                            allow_parallel_tool_calls=False
                                            )


In [7]:
response=agent_worker.chat("(77*2+2)를 78로 나눈 값을 계산해 줘")
print(response)

> Running step 5dad04ee-7217-4f64-85e4-2b70bef36007. Step input: (77*2+2)를 78로 나눈 값을 계산해 줘
Added user message to memory: (77*2+2)를 78로 나눈 값을 계산해 줘
=== Calling Function ===
Calling function: mul with args: {"a": 77, "b": 2}
=== Function Output ===
154
> Running step df5b2a9d-6737-4573-87a0-2d6c4ffc64ea. Step input: None
=== Calling Function ===
Calling function: add with args: {"a": 154, "b": 2}
=== Function Output ===
156
> Running step 9da4b6e5-f0b9-41c9-9123-0f115197c625. Step input: None
=== Calling Function ===
Calling function: div with args: {"a": 156, "b": 78}
=== Function Output ===
2.0
> Running step fc724638-7954-49c8-b87d-87b11a5fdf5b. Step input: None
=== LLM Response ===
\((77 \times 2 + 2)\)를 78로 나눈 값은 2.0입니다.
\((77 \times 2 + 2)\)를 78로 나눈 값은 2.0입니다.


## 펑션 콜링 실습 -외부 API를 활용하는 Function Calling Agent

In [8]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool
import numpy as np

llm=OpenAI(model="gpt-4o")

In [9]:
!pip install yfinance==0.2.55

import yfinance as yf

  Using cached yfinance-0.2.55-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.17.9.tar.gz (3.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached yfinance-0.2.55-py2.py3-none-any.whl (109 kB)
Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl size=139126 sha256=591b50b330c290b823af95e2b7a3d6427f505961b0b5aa11c77893d18792fa66
  Stored in directory: c:\users\hjkim\appdata\local\pip\cache\wheels\a4\4c\13\c0145c208d813977081d42e2a47d72acb3efc4a6965ab57804
Successfully built peewee


In [ ]:
#yfinance 버전 확인인
print(yf.__version__)

0.2.55


In [12]:
# 미국 주식 최신 종가 호출 
def get_stock_price_us(code):
        ticker = yf.Ticker(f"{code}")
        todays_data = ticker.history(period='1d')
        return round(todays_data['Close'].iloc[0], 2)

# 한국 주식 최신 종가 호출 
def get_stock_price_Korea(code):
        ticker = yf.Ticker(f"{code}.KS")
        todays_data = ticker.history(period='1d')
        return round(todays_data['Close'].iloc[0], 2)


In [14]:
stock_k=FunctionTool.from_defaults(fn=get_stock_price_Korea)
stock_u=FunctionTool.from_defaults(fn=get_stock_price_us)


In [15]:
agent_worker=FunctionCallingAgent.from_tools([stock_k, stock_u], 
                                            llm=llm, 
                                            verbose=True,
                                            allow_parallel_tool_calls=False
                                            )

In [16]:
response=agent_worker.chat("TESLA 최신 종가가 어떻게 돼?")


> Running step 16866750-01fc-40a8-a23c-5480bf932108. Step input: TESLA 최신 종가가 어떻게 돼?
Added user message to memory: TESLA 최신 종가가 어떻게 돼?
=== Calling Function ===
Calling function: get_stock_price_us with args: {"code": "TSLA"}
=== Function Output ===
(241.37, '2025-04-17')
> Running step baa9cb7a-3546-4e9c-9994-0f3acce60af2. Step input: None
=== LLM Response ===
TESLA의 최신 종가는 2025년 4월 17일 기준으로 $241.37입니다.


In [17]:
response=agent_worker.chat("삼성전자의 최신 종가가 어떻게 돼?")


> Running step 1027cb5b-4845-4906-b432-35bfd3c6bd85. Step input: 삼성전자의 최신 종가가 어떻게 돼?
Added user message to memory: 삼성전자의 최신 종가가 어떻게 돼?
=== Calling Function ===
Calling function: get_stock_price_Korea with args: {"code": "005930"}
=== Function Output ===
55300.0
> Running step 2a9a24af-7699-44ef-bb1d-29132617611a. Step input: None
=== LLM Response ===
삼성전자의 최신 종가는 55,300원입니다.


### 주식 최신 종가를 날짜 정보와 함께 알고 싶을 때 

In [18]:
# 종가 함수부터 아래 코드를 실행 
# 미국 주식 최신 종가를 날짜 정보와 함께 알고 싶을 때  
def get_stock_price_us(code):
        ticker = yf.Ticker(f"{code}")
        todays_data = ticker.history(period='1d')

        if not todays_data.empty:
              close_price = round(todays_data['Close'].iloc[0], 2)
              close_date = todays_data.index[0].strftime('%Y-%m-%d')
              return close_price, close_date
   
        else:
              return None, None
        

# 한국 주식 최신 종가를 날짜 정보와 함께 알고 싶을 때 
def get_stock_price_Korea(code):
        ticker = yf.Ticker(f"{code}.KS")
        todays_data = ticker.history(period='1d')

        if not todays_data.empty:
              close_price = round(todays_data['Close'].iloc[0], 2)
              close_date = todays_data.index[0].strftime('%Y-%m-%d')
              return close_price, close_date
   
        else:
              return None, None               


In [19]:
stock_k=FunctionTool.from_defaults(fn=get_stock_price_Korea)
stock_u=FunctionTool.from_defaults(fn=get_stock_price_us)

In [20]:
agent_worker=FunctionCallingAgent.from_tools([stock_k, stock_u], 
                                            llm=llm, 
                                            verbose=True,
                                            allow_parallel_tool_calls=False
                                            )

In [21]:
response=agent_worker.chat("TESLA 최신 종가가 어떻게 돼?")

> Running step 1a64fc65-d773-4e5b-b9ca-494de1685681. Step input: TESLA 최신 종가가 어떻게 돼?
Added user message to memory: TESLA 최신 종가가 어떻게 돼?
=== Calling Function ===
Calling function: get_stock_price_us with args: {"code": "TSLA"}
=== Function Output ===
(241.37, '2025-04-17')
> Running step 359dda22-c761-4885-8a54-98c00a422b0a. Step input: None
=== LLM Response ===
TESLA의 최신 종가는 2025년 4월 17일 기준으로 $241.37입니다.


In [ ]:
response=agent_worker.chat("삼성전자 최신 종가가 어떻게 돼?")

> Running step 5982dd31-2743-4019-8ab5-7d4847c84f02. Step input: 삼성전자자 최신 종가가 어떻게 돼?
Added user message to memory: 삼성전자자 최신 종가가 어떻게 돼?
=== Calling Function ===
Calling function: get_stock_price_Korea with args: {"code": "005930"}
=== Function Output ===
(55300.0, '2025-04-18')
> Running step e845d9ac-7ebd-472b-929f-ccd10addfa20. Step input: None
=== LLM Response ===
삼성전자의 최신 종가는 2025년 4월 18일 기준으로 55,300원입니다.


## 펑션 콜링 실습 - Context-Augmented Function Calling Agent

In [23]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.settings import Settings

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [ ]:

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex  
)
from llama_index.core.tools import QueryEngineTool

In [ ]:
# load data
S2024_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2024.pdf"]
    ).load_data()
S2023_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2023.pdf"]
    ).load_data()
S2022_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2022.pdf"]
    ).load_data()

# build index
S2024_index = VectorStoreIndex.from_documents(S2024_docs)
S2023_index = VectorStoreIndex.from_documents(S2023_docs)
S2022_index = VectorStoreIndex.from_documents(S2022_docs)
    

In [27]:
S2024_engine = S2024_index.as_query_engine(similarity_top_k=3)
S2023_engine = S2023_index.as_query_engine(similarity_top_k=3)
S2022_engine = S2022_index.as_query_engine(similarity_top_k=3)

In [28]:
query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=S2024_engine,
        name="samsung_2024",
        description=(
            "삼성전자의 2024년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=S2023_engine,
        name="samsung_2023",
        description=(
             "삼성전자의 2023년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=S2022_engine,
        name="samsung_2022",
        description=(
             "삼성전자의 2022년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
        ),
    ),
]

In [36]:
# agent 생성
from llama_index.core.agent import FunctionCallingAgent

agent = FunctionCallingAgent.from_tools(tools=query_engine_tools,  # tools 대신 query_engine_tools 사용
                                               llm=OpenAI(model="gpt-4o"),
                                               verbose=True,
                                               )


In [37]:
response=agent.chat("2022년 매출액과 2023년 매출액, 2024년 매출액을 차례로 알려줘")


> Running step 8f7269ce-0b55-4d08-b11b-77afd60b8f7c. Step input: 2022년 매출액과 2023년 매출액, 2024년 매출액을 차례로 알려줘
Added user message to memory: 2022년 매출액과 2023년 매출액, 2024년 매출액을 차례로 알려줘
=== Calling Function ===
Calling function: samsung_2022 with args: {"input": "2022\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2022년 삼성전자의 매출액은 302조 2,314억원입니다.
=== Calling Function ===
Calling function: samsung_2023 with args: {"input": "2023\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2023년 삼성전자의 매출액은 258조 9,355억원입니다.
=== Calling Function ===
Calling function: samsung_2024 with args: {"input": "2024\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2024년 삼성전자의 매출액은 300조 8,709억원입니다.
> Running step 75a394b2-cb51-431a-8d30-845e149498e0. Step input: None
=== LLM Response ===
- 2022년 삼성전자의 매출액은 30

In [40]:
response=agent.chat("삼성전자의 2022년, 2023년, 2024년 매출액과 영업이익을 알려줘. 그리고 2023년에는 전년대비 줄어든 매출액이 2024년에는 다시 회복된 이유를 뭐라고 분석하고 있는지 찾아줘")

> Running step f5605a40-ec7f-4375-b2b6-443893ffa36a. Step input: 삼성전자의 2022년, 2023년, 2024년 매출액과 영업이익을 알려줘. 그리고 2023년에는 전년대비 줄어든 매출액이 2024년에는 다시 회복된 이유를 뭐라고 분석하고 있는지 찾아줘
Added user message to memory: 삼성전자의 2022년, 2023년, 2024년 매출액과 영업이익을 알려줘. 그리고 2023년에는 전년대비 줄어든 매출액이 2024년에는 다시 회복된 이유를 뭐라고 분석하고 있는지 찾아줘
=== Calling Function ===
Calling function: samsung_2022 with args: {"input": "2022\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uacfc \uc601\uc5c5\uc774\uc775\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2022년 삼성전자의 매출액은 302조 2,314억 원이며, 영업이익은 51,633,856백만 원입니다.
=== Calling Function ===
Calling function: samsung_2023 with args: {"input": "2023\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uacfc \uc601\uc5c5\uc774\uc775\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2023년 삼성전자의 매출액은 258조 9,355억원입니다. 영업이익에 대한 정보는 제공된 자료에 포함되어 있지 않습니다.
=== Calling Function ===
Calling function: samsung_2024 with args: {"input": "2024\ub144 \uc0bc\uc131